<a href="https://colab.research.google.com/github/vrramakanth/capstone_project_practice/blob/main/Capstone_Phase2_Project_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
from scipy.stats import zscore
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


air_traffic_df = pd.read_csv('/content/Air_Traffic_Passenger_Statistics.csv')
airplane_crashes_df = pd.read_csv('/content/Airplane_Crashes_and_Fatalities_Since_1908.csv')

air_traffic_df = air_traffic_df.dropna()
airplane_crashes_df = airplane_crashes_df.dropna()

air_traffic_df = air_traffic_df.drop_duplicates()
airplane_crashes_df = airplane_crashes_df.drop_duplicates()

# Remove outliers using Z-score
def remove_outliers_zscore(df, threshold=3):
    numeric_cols = df.select_dtypes(include=np.number).columns
    for col in numeric_cols:
        df[col + '_zscore'] = np.abs(zscore(df[col]))
        df = df[df[col + '_zscore'] <= threshold]
        df = df.drop(columns=[col + '_zscore'])
    return df

air_traffic_df = remove_outliers_zscore(air_traffic_df)
airplane_crashes_df = remove_outliers_zscore(airplane_crashes_df)

# Extract the year from the 'Date' column
#airplane_crashes_df['Year'] = pd.to_datetime(airplane_crashes_df['Date'], format='%m/%d/%Y').dt.year

# Merge the dataframes on 'Year'
#merged_df = pd.merge(airplane_crashes_df, air_traffic_df, on='Year')
#-------FROM PHASE 1 PROJECT----
airplane_crashes_df['Year'] = pd.to_datetime(airplane_crashes_df['Date']).dt.year
#df_fatalities_yearly = airplane_crashes_df.groupby('Year')['Fatalities'].sum().reset_index()
merged_df = pd.merge(airplane_crashes_df, air_traffic_df, on='Year')
#-------TILL HERE----

#print("Datasets merged successfully on Year.")
#print("Air Traffic DataFrame after removing NA, duplicates and outliers:")
#display(air_traffic_df.head())
#print("\nAir Traffic DataFrame Info after removing NA, duplicates and outliers:")
#air_traffic_df.info()

#print("\nAirplane Crashes DataFrame after removing NA, duplicates and outliers:")
#display(airplane_crashes_df.head())
#print("\nAirplane Crashes DataFrame Info after removing NA, duplicates and outliers:")
#airplane_crashes_df.info()

print("\nMerged DataFrame:")
display(merged_df.head())
print("Total Number of rows=", len(merged_df))
print("Total No. of columns/features =" , len(merged_df.columns))
print("Name of columns :" , merged_df.columns.tolist())
print(merged_df.info())
print(merged_df.describe())


correlation_fatalities_passenger = merged_df['Fatalities'].corr(merged_df['Passenger Count'])
print(f"The correlation between Fatalities and Passenger Count is: {correlation_fatalities_passenger}")
correlation_fatalities_year = merged_df['Fatalities'].corr(merged_df['Year'])
print(f"The correlation between Fatalities and Year is: {correlation_fatalities_year}")

categorical_cols = merged_df.select_dtypes(include=['object', 'category']).columns.tolist()
print("Categorical variables in merged_df:")
print(categorical_cols)

# Encode categorical variables using one-hot encoding
merged_df_encoded = pd.get_dummies(merged_df, columns=categorical_cols)
print("\nEncoded DataFrame:")
display(merged_df_encoded.head())
print("\nEncoded DataFrame Info:")
merged_df_encoded.info()

merged_df_encoded['Fatalities_Binary'] = (merged_df_encoded['Fatalities'] > 0).astype(int)
display(merged_df_encoded[['Fatalities', 'Fatalities_Binary']])

fatalities_counts = merged_df_encoded['Fatalities_Binary'].value_counts()

print("Count of rows with Fatalities_Binary = 1:", fatalities_counts.get(1, 0))
print("Count of rows with Fatalities_Binary = 0:", fatalities_counts.get(0, 0))



# Define features (independent variables) and target (dependent variable)
features = ['Year', 'Adjusted Passenger Count']
target = 'Fatalities'

X = merged_df[features]
y = merged_df[target]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse) # Calculate RMSE by taking the square root of MSE
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")

# Display the coefficients
print("\nModel Coefficients:")
for feature, coef in zip(features, model.coef_):
    print(f"{feature}: {coef}")

print(f"\nIntercept: {model.intercept_}")



# Define features (independent variables) and target (dependent variable)
features = [col for col in merged_df_encoded.columns if 'Published Airline_' in col or 'GEO Region_' in col]
target = 'Fatalities'

X = merged_df_encoded[features]
y = merged_df_encoded[target]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse) # Calculate RMSE by taking the square root of MSE
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")

# Display the coefficients
print("\nModel Coefficients:")
for feature, coef in zip(features, model.coef_):
    print(f"{feature}: {coef}")

print(f"\nIntercept: {model.intercept_}")


Merged DataFrame:


,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,...,GEO Summary,GEO Region,Activity Type Code,Price Category Code,Terminal,Boarding Area,Passenger Count,Adjusted Activity Type Code,Adjusted Passenger Count,Month
0,02/03/2005,17:00,"Near Peshawar, Pakistan",Kam Air,904,Herat - Kabul,Boeing B-737-200,EX-037,22075/630,104.0,...,Domestic,US,Deplaned,Low Fare,Terminal 1,B,27271,Deplaned,27271,July
1,02/03/2005,17:00,"Near Peshawar, Pakistan",Kam Air,904,Herat - Kabul,Boeing B-737-200,EX-037,22075/630,104.0,...,Domestic,US,Enplaned,Low Fare,Terminal 1,B,29131,Enplaned,29131,July
2,02/03/2005,17:00,"Near Peshawar, Pakistan",Kam Air,904,Herat - Kabul,Boeing B-737-200,EX-037,22075/630,104.0,...,Domestic,US,Thru / Transit,Low Fare,Terminal 1,B,5415,Thru / Transit * 2,10830,July
3,02/03/2005,17:00,"Near Peshawar, Pakistan",Kam Air,904,Herat - Kabul,Boeing B-737-200,EX-037,22075/630,104.0,...,International,Canada,Deplaned,Other,Terminal 1,B,35156,Deplaned,35156,July
4,02/03/2005,17:00,"Near Peshawar, Pakistan",Kam Air,904,Herat - Kabul,Boeing B-737-200,EX-037,22075/630,104.0,...,International,Canada,Enplaned,Other,Terminal 1,B,34090,Enplaned,34090,July


Total Number of rows= 56082
Total No. of columns/features = 29
Name of columns : ['Date', 'Time', 'Location', 'Operator', 'Flight #', 'Route', 'Type', 'Registration', 'cn/In', 'Aboard', 'Fatalities', 'Ground', 'Summary', 'Year', 'Activity Period', 'Operating Airline', 'Operating Airline IATA Code', 'Published Airline', 'Published Airline IATA Code', 'GEO Summary', 'GEO Region', 'Activity Type Code', 'Price Category Code', 'Terminal', 'Boarding Area', 'Passenger Count', 'Adjusted Activity Type Code', 'Adjusted Passenger Count', 'Month']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56082 entries, 0 to 56081
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         56082 non-null  object 
 1   Time                         56082 non-null  object 
 2   Location                     56082 non-null  object 
 3   Operator                     56082 non-null  object 
 

,Aboard,Fatalities,Ground,Year,Activity Period,Passenger Count,Adjusted Passenger Count,Date_01/01/2007,Date_01/15/2009,Date_01/25/2007,...,Month_December,Month_February,Month_January,Month_July,Month_June,Month_March,Month_May,Month_November,Month_October,Month_September
0,104.0,104.0,0.0,2005,200507,27271,27271,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,104.0,104.0,0.0,2005,200507,29131,29131,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,104.0,104.0,0.0,2005,200507,5415,10830,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,104.0,104.0,0.0,2005,200507,35156,35156,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,104.0,104.0,0.0,2005,200507,34090,34090,False,False,False,...,False,False,False,True,False,False,False,False,False,False



Encoded DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56082 entries, 0 to 56081
Columns: 740 entries, Aboard to Month_September
dtypes: bool(733), float64(3), int32(1), int64(3)
memory usage: 42.0 MB


,Fatalities,Fatalities_Binary
0,104.0,1
1,104.0,1
2,104.0,1
3,104.0,1
4,104.0,1
...,...,...
56077,11.0,1
56078,11.0,1
56079,11.0,1
56080,11.0,1


Count of rows with Fatalities_Binary = 1: 52715
Count of rows with Fatalities_Binary = 0: 3367
Mean Squared Error (MSE): 2811.4598514415306
Root Mean Squared Error (RMSE): 53.02320106747169
R-squared (R2): 0.052129251577280655

Model Coefficients:
Year: -9.75610760383673
Adjusted Passenger Count: 1.520031830621949e-06

Intercept: 19629.753292636135
Mean Squared Error (MSE): 2953.3481990608357
Root Mean Squared Error (RMSE): 54.344716385871735
R-squared (R2): 0.004292248256242415

Model Coefficients:
Published Airline_ATA Airlines: 6.782694240714447
Published Airline_Aer Lingus: -11.751181922187406
Published Airline_Aeromexico: -29.207166074727663
Published Airline_Air Canada : 2.3740177183855815
Published Airline_Air China: 0.2064435454521698
Published Airline_Air France: 0.7645712294495324
Published Airline_Air New Zealand: 0.4403801155895657
Published Airline_AirTran Airways: 3.0944805418844474
Published Airline_Alaska Airlines: 3.0786851528891868
Published Airline_All Nippon Airways